In [1]:
import pandas as pd
import numpy as np
from plotly import express as px

In [2]:
churn_df = pd.read_csv('data/telecom_customer_churn.csv')

In [3]:
churn_df.head(10)

,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.60,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.00,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.90,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.00,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.90,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability
5,0013-MHZWF,Female,23,No,3,Midpines,95345,37.581496,-119.972762,0,...,Credit Card,69.40,571.45,0.00,0,150.93,722.38,Stayed,NaN,NaN
6,0013-SMEOE,Female,67,Yes,0,Lompoc,93437,34.757477,-120.550507,1,...,Bank Withdrawal,109.70,7904.25,0.00,0,707.16,8611.41,Stayed,NaN,NaN
7,0014-BMAQU,Male,52,Yes,0,Napa,94558,38.489789,-122.270110,8,...,Credit Card,84.65,5377.80,0.00,20,816.48,6214.28,Stayed,NaN,NaN
8,0015-UOCOJ,Female,68,No,0,Simi Valley,93063,34.296813,-118.685703,0,...,Bank Withdrawal,48.20,340.35,0.00,0,73.71,414.06,Stayed,NaN,NaN
9,0016-QLJIS,Female,43,Yes,1,Sheridan,95681,38.984756,-121.345074,3,...,Credit Card,90.45,5957.90,0.00,0,1849.90,7807.80,Stayed,NaN,NaN


In [4]:
print('Number of columns (or features): ', len(churn_df.columns))
print('Number of rows: ', len(churn_df))

Number of columns (or features):  38
Number of rows:  7043


In [5]:
for col in churn_df.columns:
    print(col)

Customer ID
Gender
Age
Married
Number of Dependents
City
Zip Code
Latitude
Longitude
Number of Referrals
Tenure in Months
Offer
Phone Service
Avg Monthly Long Distance Charges
Multiple Lines
Internet Service
Internet Type
Avg Monthly GB Download
Online Security
Online Backup
Device Protection Plan
Premium Tech Support
Streaming TV
Streaming Movies
Streaming Music
Unlimited Data
Contract
Paperless Billing
Payment Method
Monthly Charge
Total Charges
Total Refunds
Total Extra Data Charges
Total Long Distance Charges
Total Revenue
Customer Status
Churn Category
Churn Reason


In [6]:
print('Number of unique cities: ',churn_df['City'].nunique())

Number of unique cities:  1106


In [7]:
# print the 5 cities with the most customers
churn_df['City'].value_counts().head(10)

Los Angeles      293
San Diego        285
San Jose         112
Sacramento       108
San Francisco    104
Fresno            61
Long Beach        60
Oakland           52
Escondido         51
Stockton          44
Name: City, dtype: int64

In [8]:
# map the values of the feature 'Customer Status' to numerical values
churn_df['Customer Status'] = churn_df['Customer Status'].map({'Stayed': 1, 'Churned': 0})

In [9]:
# check the number of missing values in each column
churn_df.isnull().sum()

Customer ID                             0
Gender                                  0
Age                                     0
Married                                 0
Number of Dependents                    0
City                                    0
Zip Code                                0
Latitude                                0
Longitude                               0
Number of Referrals                     0
Tenure in Months                        0
Offer                                   0
Phone Service                           0
Avg Monthly Long Distance Charges     682
Multiple Lines                        682
Internet Service                        0
Internet Type                        1526
Avg Monthly GB Download              1526
Online Security                      1526
Online Backup                        1526
Device Protection Plan               1526
Premium Tech Support                 1526
Streaming TV                         1526
Streaming Movies                  

In [ ]:
# drop irelevant features
churn_df_clean = churn_df.drop(['Customer ID','Latitude', 'Longitude', 'Churn Label'], axis=1, inplace=True).copy()

In [ ]:
# create a function that calculates the weight of evidence of each category in a feature
#  add information value of the feature
def calc_weight_of_evidence(df, target):
    df[f'Good'] = np.where(df[target] == 0, 1, 0)
    df[f'Bad'] = np.where(df[target] == 1, 1, 0)
    total_good = df['Good'].sum()
    total_bad = df['Bad'].sum()
    iv = {}
    for feature in df.columns:
        # ignore the target feature
        if feature == target or feature == 'Good' or feature == 'Bad' or feature in ['Latitude', 'Longitude', 'Zip Code', 'City', 'State',]:
            continue
        grouped = df.groupby(feature).agg({'Good': 'sum', 'Bad': 'sum'})
        grouped['DistributionGood'] = grouped['Good'] / total_good
        grouped['DistributionBad'] = grouped['Bad'] / total_bad
        grouped['WoE'] = np.log(grouped['DistributionGood'] / grouped['DistributionBad'])
        woe_dict = grouped['WoE'].to_dict()
        information_value = ((grouped['DistributionGood'] - grouped['DistributionBad']) * grouped['WoE']).sum()
        df[feature] = df[feature].map(woe_dict)
        iv[feature] = information_value
    #  if iv is less than 0.02, then the feature is not useful for prediction, drop it
    for feature in churn_df.columns:
        if iv[feature] < 0.02:
            print(f'Dropping {feature} with IV {iv[feature]}''')
            df.drop(feature, axis=1, inplace=True)
    return df

In [ ]:
# Example usage
churn_df_post_iv = calc_weight_of_evidence(churn_df, 'Customer Status')

In [ ]:
churn_df_post_iv.head()

In [ ]:
# use px.density_mapbox to show the number of customers in each city, center the map on California
fig = px.density_mapbox(churn_df, lat='Latitude', lon='Longitude', radius=10, zoom=5.5, mapbox_style='stamen-terrain', height=1200, width=900)
fig.show()

In [ ]:
# in churn df find all the features that are catigorical
categorical_features = [col for col in churn_df.columns if churn_df[col].dtype == 'object']

In [ ]:
categorical_features

In [ ]:
# use PCA  for churn df
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# create a copy of the churn df
churn_df_pca = churn_df.copy()

# drop the target feature
churn_df_pca.drop('Customer Status', axis=1, inplace=True)

# drop the categorical features
churn_df_pca.drop(categorical_features, axis=1, inplace=True)

# standardize the data
scaler = StandardScaler()
churn_df_pca = scaler.fit_transform(churn_df_pca)

# create a PCA object
pca = PCA(n_components=2)

# fit the PCA object
pca.fit(churn_df_pca)

# transform the data
churn_df_pca = pca.transform(churn_df_pca)

# create a dataframe with the PCA data
churn_df_pca = pd.DataFrame(churn_df_pca, columns=['PC1', 'PC2'])

# add the target feature to the dataframe
churn_df_pca['Customer Status'] = churn_df['Customer Status']

